In [1]:
import os
path='/home/ubuntu/workspace/t1/'
pypath='/home/ubuntu/workspace/t1/PythonCode'
os.chdir(path)
import django
django.setup()
os.chdir(pypath)
from openpyxl import load_workbook, Workbook, cell
import pandas as pd, numpy as np
from prof.models import FileSpace, RunSpace, TransactionData, PnLData
from prof.settings import PnL_settings_dict
from prof.ProcessingFunctions import ifHeaderReturnIndex, isRowValid, isRevAggregate, isRowAggregate, processPnLSheet_to_pd
from prof.ProcessingFunctions import returnPnLPeriodColumnNames


def processPnLFile_to_file(inputFile, sheetName, outputFile, desiredColumns, revenueMarkers):
    '''This function is deprecated
    This function processes a PnL file. It takes an input file name, and then writes to an output file name
    Both input and output are excel files. The arguments are
    1. inputFile: Name of the input excel file
    2. sheetName: Name of the worksheet to process
    3. outputFile: Name of the output excel file
    4. desiredColumns: The header row
    5. revenueMarkers: A list of strings that should be interpreted as marking Net Revenue'''
    wb=load_workbook(inputFile, read_only=True, data_only=True)
    ws=wb.get_sheet_by_name(sheetName)
    output_wb=Workbook()
    output_ws=output_wb.active
    output_ws.title=sheetName
    revenueSwitch=True
    for row in ws.iter_rows():
        if ifHeaderReturnIndex(row,desiredColumns):
            selectedIndex=ifHeaderReturnIndex(row, desiredColumns)
            outputColumnNames=[row[i].value for i in selectedIndex]
            outputColumnNames.append('Aggregate')
            outputColumnNames.append('RevorCost')
            output_ws.append(outputColumnNames)
            break
    for row in ws.iter_rows(min_row=2):
        if ifHeaderReturnIndex(row,desiredColumns):
            pass
        elif isRowValid(row,selectedIndex):
            newrow=[row[i].value for i in selectedIndex]
            newrow.append(isRowAggregate(row,selectedIndex))
            newrow.append(revenueSwitch)
            if isRevAggregate(row,selectedIndex): revenueSwitch = False
            output_ws.append(newrow)
    output_wb.save(outputFile)
    return(outputFile)


def createGroupfileFromFrame(PnLframe, outputFile):
    PnLframe['Group']=None
    for index,row in PnLframe.iterrows():
        if row['Aggregate']==True: PnLframe.set_value(index,'Group','Not Applicable')
    groupAllocationFrame=PnLframe[['Line Description', 'Line Code', 'Group']][~PnLframe['RevorCost']]
    groupAllocationFrame.to_excel(outputFile, index=False)

def collapsePnLbyGroups(PnLframe,groupFile):
    groups=pd.read_excel(groupFile)
    return pd.merge(PnLframe,groups,how='outer', on=['Line Description', 'Line Code']).groupby(['Business Unit', 'Group']).sum()
    

In [2]:
run=RunSpace.objects.get(pk=1)
file= FileSpace.objects.get(FileName='SamplePnL')
#TrFile=FileSpace.objects.get(FileName='Filled Sample')
#wb=load_workbook(TrFile.File, read_only=True, data_only=True)


In [3]:
returnPnLPeriodColumnNames(run)

['Jan-2016',
 'Feb-2016',
 'Mar-2016',
 'Apr-2016',
 'May-2016',
 'Jun-2016',
 'Jul-2016',
 'Aug-2016',
 'Sep-2016',
 'Oct-2016',
 'Nov-2016',
 'Dec-2016']

In [4]:
inputFile='SamplePnL.xlsx'
wb=load_workbook(inputFile, read_only=True, data_only=True)
ws=wb.get_sheet_by_name('BU-1')    
#processPnLSheet(ws, desiredColumns, revenueMarkers)
 

fullPnL=pd.concat([processPnLSheet_to_pd(ws, run).set_index(['Line Description', 'Line Code']) for ws in wb.worksheets], keys=wb.get_sheet_names(),names=['Business Unit']).reset_index().replace(np.nan,0)
fullPnL    

,Business Unit,Line Description,Line Code,Jan-2016,Feb-2016,Mar-2016,Apr-2016,May-2016,Jun-2016,Jul-2016,Aug-2016,Sep-2016,Oct-2016,Nov-2016,Dec-2016,All Months,RevorCost,Aggregate
0,PnLData,Sales,R10001014,1.083254e+08,9.266333e+07,1.078674e+08,1.106778e+08,1.035081e+08,1.097036e+08,1.189169e+08,1.059081e+08,9.744222e+07,9.944987e+07,1.010836e+08,1.063723e+08,1.261919e+09,False,True
1,PnLData,Sales Returns (Reduction),R10001015,-9.242405e+04,-3.820668e+04,-4.786577e+05,-6.368089e+03,-3.745589e+05,-1.280370e+05,-9.371903e+04,0.000000e+00,-1.708225e+05,0.000000e+00,-4.420780e+03,-1.008918e+04,-1.397304e+06,False,True
2,PnLData,Sales Discounts (Reduction),R10001016,-1.271143e+06,-1.342104e+06,-1.259850e+06,-1.267535e+06,-1.292270e+06,-1.331649e+06,-1.479435e+06,-1.138542e+06,-1.226452e+06,-1.278029e+06,-1.460392e+06,-1.440278e+06,-1.578768e+07,False,True
3,PnLData,Other Revenue 1,R10001017,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,False,True
4,PnLData,Other Revenue 2,R10001018,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,False,True
5,PnLData,Other Revenue 3,R10001019,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,False,True
6,PnLData,…,R10001020,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,False,True
7,PnLData,Net Revenue,0,1.069619e+08,9.128302e+07,1.061289e+08,1.094039e+08,1.018412e+08,1.082439e+08,1.173437e+08,1.047696e+08,9.604495e+07,9.817184e+07,9.961878e+07,1.049219e+08,1.244734e+09,True,True
8,PnLData,Material 1,C10001023,8.964260e+06,1.642466e+07,1.651836e+07,1.401046e+07,8.626340e+06,8.697019e+06,1.371466e+07,1.675444e+07,1.494206e+07,9.698396e+06,1.121024e+07,1.085537e+07,1.504163e+08,False,False
9,PnLData,Material 2,C10001024,1.234231e+07,1.416853e+07,8.507249e+06,8.593166e+06,1.404401e+07,9.535521e+06,1.107182e+07,7.023561e+06,1.186157e+07,1.140811e+07,8.915638e+06,8.500082e+06,1.259716e+08,False,False


In [6]:
periodColumns = returnPnLPeriodColumnNames(run)
desiredColumns = PnL_settings_dict['leading_column_names'] + returnPnLPeriodColumnNames(run) + [PnL_settings_dict['total_column_name']]
columnsForModel = PnL_settings_dict['leading_column_names'] +[PnL_settings_dict['revenue_or_cost_column_name']] + [PnL_settings_dict['aggregate_line_flag_column_name']]

fullPnL = pd.melt(fullPnL,id_vars=columnsForModel, value_vars=       
    periodColumns,
        var_name = PnL_settings_dict['period_column_name'], value_name=PnL_settings_dict['amount_column_name'])


In [7]:
fullPnL

,Line Description,Line Code,RevorCost,Aggregate,Period,Amount
0,Sales,R10001014,False,True,Jan-2016,1.083254e+08
1,Sales Returns (Reduction),R10001015,False,True,Jan-2016,-9.242405e+04
2,Sales Discounts (Reduction),R10001016,False,True,Jan-2016,-1.271143e+06
3,Other Revenue 1,R10001017,False,True,Jan-2016,0.000000e+00
4,Other Revenue 2,R10001018,False,True,Jan-2016,0.000000e+00
5,Other Revenue 3,R10001019,False,True,Jan-2016,0.000000e+00
6,…,R10001020,False,True,Jan-2016,0.000000e+00
7,Net Revenue,0,True,True,Jan-2016,1.069619e+08
8,Material 1,C10001023,False,False,Jan-2016,8.964260e+06
9,Material 2,C10001024,False,False,Jan-2016,1.234231e+07
